In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from PIL import Image, ImageFilter, ImageOps
import pandas as pd

def save_heatmap(grid, cmap, filename, size=1300):
    fig, ax = plt.subplots(figsize=(size / 300, size / 300), dpi=300)
    ax.imshow(grid, cmap=cmap, origin='lower')
    ax.axis('off')
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    plt.savefig(filename, dpi=300)
    plt.close()

In [2]:
bu_colormaps = [
    ["#FFFFFF", "#CC0000", "#4B0000"],  # classic white → scarlet → deep red
    ["#FFF0F0", "#D32F2F", "#660000"],  # soft white → vivid red → dark maroon
    ["#FFCCCC", "#B71C1C", "#330000"],  # pastel → maroon → near-black
    ["#FFE5E5", "#E53935", "#440000"],  # rosy white → coral red → charcoal
    ["#FFF5F5", "#C62828", "#2D0000"],  # milky pink → deep scarlet → blackened red
    ["#FCE4EC", "#AD1457", "#3B0000"],  # rose white → crimson pink → blood red
    ["#FFF8F8", "#B71C1C", "#1A0000"],  # faint rose → BU red → pitch red
    ["#F8BBD0", "#C2185B", "#4A0000"],  # bubblegum → wine → ember
    ["#FAFAFA", "#C62828", "#0F0000"],  # pale light → bright scarlet → ultra dark
    ["#FFEBEE", "#D50000", "#5E0000"],  # soft rose → BU red max → cherry coal
    ["#FFF0F0", "#C51162", "#3A0000"],  # faded blush → hot pink-red → smoked burgundy
    ["#FFEBEE", "#E53935", "#300000"],  # classic red → rich scarlet → muted black
    ["#FDEDEC", "#FF1744", "#440000"],  # candy tint → fluorescent scarlet → wine black
    ["#FDF2F3", "#FF0033", "#520000"],  # cloudy rose → aggressive red → lava stone
    ["#F9D6D5", "#B00020", "#280000"],  # dusty blush → maroon fire → void red
    ["#FADBD8", "#A93226", "#3C0000"],  # creamy coral → rust red → obsidian blood
]


In [3]:
import os
os.mkdir('imgs')

FileExistsError: [Errno 17] File exists: 'imgs'

In [6]:
n=3
for idx in range(n**2):
    j,i = divmod(idx,n) #j is columns; i is rows
    v= vals2
    if j==i or j+i==n-1:
        v = vals
    cmap = LinearSegmentedColormap.from_list(f"bu_{idx}", bu_colormaps[idx])
    save_heatmap(v,cmap,f'imgs/tile_{idx}')

In [5]:
vals = np.array(pd.read_csv('divergence_heatmap.csv'))
vals2 = np.array(pd.read_csv('divergence_heatmap_2.csv'))

In [7]:

def apply_soft_edges(image, fade_size=100):
    # Create an alpha mask with faded edges
    width, height = image.size
    mask = Image.new('L', (width, height), 255)
    fade = Image.linear_gradient('L').resize((fade_size, height))

    # Left fade
    mask.paste(fade, (0, 0))
    # Right fade (flipped)
    mask.paste(fade.transpose(Image.FLIP_LEFT_RIGHT), (width - fade_size, 0))
    
    # Top fade
    fade_t = Image.linear_gradient('L').resize((width, fade_size))
    mask.paste(fade_t, (0, 0), fade_t)
    # Bottom fade (flipped)
    mask.paste(fade_t.transpose(Image.FLIP_TOP_BOTTOM), (0, height - fade_size), fade_t.transpose(Image.FLIP_TOP_BOTTOM))

    # Convert image to RGBA and apply mask
    image = image.convert("RGBA")
    image.putalpha(mask)
    return image


In [8]:
n = 3
tile_size = 1300
canvas_size = tile_size * n

final_image = Image.new("RGBA", (canvas_size, canvas_size), (255, 255, 255, 255))

for idx in range(n * n):
    row, col = divmod(idx, n)
    tile_path = f"imgs/tile_{idx}.png"
    img = Image.open(tile_path).resize((tile_size, tile_size))
    img = apply_soft_edges(img)

    final_image.alpha_composite(img, dest=(col * tile_size, row * tile_size))

final_image.convert("RGB").save("final_img.jpg", format="JPEG", quality=95,dpi=(300,300))

